# EJERCICIOS

In [1]:
import warnings
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier,StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

## EJERCICIO 1

Si has entrenado cinco modelos diferentes en el mismo conjunto de entrenamiento exacto y todos consiguen una precisión del 95%, ¿hay alguna posibilidad de que puedas combinar estos modelos para obtener mejores resultados? 

Si la respuesta es sí, ¿cómo? Si la respuesta es no, ¿por qué?

In [3]:
# Si has entrenado cinco modelos diferentes y todos alcanzan una precisión del 95%, puedes intentar combinarlos en un conjunto de votación, que a menudo dará resultados aún mejores. 

# Funciona mejor si los modelos son muy diferentes (por ejemplo, un clasificador SVM, un clasificador de árbol de decisión, un clasificador de regresión logística, etc.). Es incluso mejor si están entrenados en diferentes instancias de entrenamiento (ese es el objetivo de ensamblar y pegar conjuntos), pero si no, esto seguirá siendo eficaz siempre que los modelos sean muy diferentes.

## EJERCICIO 2

Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50.000 instancias para entrenamiento, 10.000 para validación y 10.000 para pruebas). 

Después, entrena varios clasificadores diferentes (uno de ellos que sea un árbol de decisión). 

A continuación, intenta combinarlos en un ensamble que supere en rendimiento a cada clasificador individual del conjunto de validación, utilizando hard voting. 

Una vez que hayas encontrado uno, pruébalo en el conjunto de pruebas.

In [3]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")

#DIVIDIR LOS DATOS COMO DICE EL ENUNCIADO.
X_mnist, y_mnist = mnist.data, mnist.target
#DATOS ENTRENAMIENTO
X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]
#DATOS VALIDACION
X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]
#DATOS VALIDACION
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=42)
lr_clf = LogisticRegression(random_state=42)
svm_clf = SVC(random_state=42)
knn_clf = KNeighborsClassifier()

estimators = [dt_clf, lr_clf, svm_clf, knn_clf]
for estimator in estimators:
    print("Entrenando ", estimator)
    estimator.fit(X_train, y_train)

In [5]:
[estimator.score(X_valid, y_valid) for estimator in estimators]

[0.8822, 0.9272, 0.9802, 0.9718]

In [ ]:
named_estimators = [
    ("dt_clf", dt_clf),
    ("lr_clf", lr_clf),
    ("svm_clf", svm_clf),
    ("knn_clf", knn_clf)
]

voting_clf = VotingClassifier(named_estimators)
error_train=voting_clf.fit(X_train, y_train)
error_val=voting_clf.score(X_valid, y_valid)
print("Precisión del conjunto de validación: ", error_val)

In [ ]:
voting_clf.score(X_test, y_test)

## EJERCICIO 3

Ejecuta los clasificadores individuales del ejercicio anterior para hacer predicciones en el conjunto de entrenamiento y crea un nuevo conjunto de entrenamiento con las predicciones resultantes: cada instancia de entrenamiento es un vector que contiene el conjunto de predicciones de todos tus clasificadores para una imagen y el objetivo es la clase de la imagen. Entrena un clasificador (RandomForestClassifier) en este nuevo conjunto de entrenamiento. 

Acabas de entrenar un blender y, junto a los clasificadores, forma un ensamble de stacking.

Ahora, evalúa el ensamble en el conjunto de prueba. 

¿Cómo es en comparación con el clasificador de votación que has entrenado antes?

Haz lo mismo usando StackingClassifier

In [9]:
X_train_predictions = np.empty((len(X_train), len(estimators)), dtype=object)

for index, estimator in enumerate(estimators):
    X_train_predictions[:, index] = estimator.predict(X_train)

In [10]:
X_train_predictions

array([['5', '5', '5', '5'],
       ['0', '0', '0', '0'],
       ['4', '4', '4', '4'],
       ...,
       ['8', '8', '8', '8'],
       ['4', '4', '4', '4'],
       ['8', '8', '8', '8']], dtype=object)

In [11]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, random_state=42)
rnd_forest_blender.fit(X_train_predictions, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [12]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=object)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [13]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [14]:
accuracy_score(y_test, y_pred)

0.8694

In [15]:
stack_clf = StackingClassifier(named_estimators,final_estimator=rnd_forest_blender)
stack_clf.fit(X_train, y_train)

/Users/ainara/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ainara/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

StackingClassifier(estimators=[('dt_clf',
                                DecisionTreeClassifier(random_state=42)),
                               ('lr_clf', LogisticRegression(random_state=42)),
                               ('svm_clf', SVC(random_state=42)),
                               ('knn_clf', KNeighborsClassifier())],
                   final_estimator=RandomForestClassifier(n_estimators=200,
                                                          random_state=42))

In [17]:
stack_clf.score(X_test, y_test)

0.9809